In [58]:
import torch
import os 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
import mlflow
import mlflow.pytorch

In [59]:
class Config:
    EPOCHS=10
    BATCH_SIZE=32
    LR=0.01
    DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
    GAMA=0.7 #->Associate with step lr.. after certain value of steps this gamma value will be multimlied with learning rate.. so that further reducing the LR
    SEED=42
    log_interval=10 # how many intervals you want to log the outputs or print the outcomes.
    TEST_BATCH_SIZE=100
    DRY_RUN=True # just to check you are going through entire training or the function you implement just once without iterating over the entire 

In [60]:
config=Config()

In [61]:
# Define the Neural Network
class ConvNet(nn.Module): # inherite the neural network module whenever you will create your network
    # define constructor
    def __init__(self):
        #import super class, all the possible constructor are there or the function which are used or argument which can be initialized as a parent network.

        super(ConvNet, self).__init__()
        #define convolution layer
        self.conv1=nn.Conv2d(1, 32,3,1)# 1->input image, 32->filters 3->kernel, 1->strides
        self.conv2=nn.Conv2d(32,64,3,1)
        self.dropout1=nn.Dropout(0.25)
        self.dropout2=nn.Dropout(0.5)
        self.fc1=nn.Linear(9216, 128)
        self.fc2=nn.Linear(128,10)
    
    
    def forword(self,x):# x->data set training data
        # it will connect with the convolution layer one and it will take the x as an input to the convolution layer one
        x=self.conv1(x)
        x=F.relu(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=F.max_pool2d(x,2)# 2->stride 2
        x=self.dropout1(x)
        x=torch.flatten(x,1)# 1->one dimension
        x=self.fc1(x)
        x=F.relu(x)
        x=self.dropout2(x)
        x=self.fc2(x)
        
        output=F.log_softmax(x, dim=1)
        return output
        
        
    
    

In [62]:
def train(config, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target=data.to(device), data.to(device)# bring the data into device either cpu or 
        # before using optimizer we have to zero the gradient if there is any residual gradient from the past iteration
        
        optimizer.zero_grad()
        # calculate the prediction outcome
        pred=model(data)# it will take the model and it will take the data
        #Calculate the loss with cross entropy loss which will take the pred and actual value that is the target
        loss=F.cross_entropy(pred, actual)
        #once the loss is getting calculated loss function should be->it is actually we take a differentiaon of it or gradient of it
        loss.backward()
        # after calculating the gradient we can go for optimizer
        optimizer.step()# weight update
        #Now we can log the values or print the values
        if batch_idx % config.log_interval==0:
            # multiplied by batch_idx the length of the data. data->data out of the train_loader which are passing and then we are going to write the out of entire data point--trainloader.datasets
            #
            print(f"train epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 *batch_idx / len(train_loader):.0f})]\t Loss:  {loss.item():.6f}")
            if config.DRY_RUN:
                break

        

In [63]:
def train_(config, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = F.cross_entropy(pred, target)
        loss.backward()
        optimizer.step()
        if batch_idx % config.LOG_INTERVAL == 0:
            print(f"train epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 *batch_idx / len(train_loader):.0f})]\t Loss:  {loss.item():.6f}")
            
            if config.DRY_RUN:
                break

In [64]:
def test(model, device, test_loader):
    pass

In [65]:
torch.manual_seed(config.SEED)

In [66]:
# define keywords arguments
train_kwargs={"batch_size": config.BATCH_SIZE}
print(train_kwargs)
test_kwargs={"batch_size": config.TEST_BATCH_SIZE}


{'batch_size': 32}


In [67]:
if config.DEVICE=="cuda":
    cuda_kwargs={"run_workers":1, "pin_memory":True, "suffle":True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [68]:
transforms=transforms.Compose(
    [transforms.ToTensor()]# convert image to Tensor.. numpy aray to Tensor array

)

In [70]:
train=datasets.MNIST("../data", train=True, download=True, transform=transforms)
test=datasets.MNIST("../data", train=False, transform=transforms)

train_loader=torch.utils.data.DataLoader(train, **train_kwargs)
test_loader=torch.utils.data.DataLoader(test, **test_kwargs)


In [71]:
#define the model
model=ConvNet().to(config.DEVICE)#ConvNet putting into the device or cuda
# pass the parameters of the model because adam is an optimizer which works on the model parameters
#model parameters are actually gets updated here.. all the trainable and parameters will be updated
optimizer=torch.optim.Adam(model.parameters(), lr=config.LR)
scheduler=StepLR(optimizer, step_size=1, gamma=config.GAMA)

In [42]:
# training loop
for epoch in range(1, config.EPOCHS+1):
    train(config, model, config.DEVICE, train_loader, optimizer, epoch)
    # Every steps after training is passed we are going to take scheduler steps..that means we will be multiplying random with the gamma
    scheduler.step()
    

NotImplementedError: 

In [72]:
# training loop
for epoch in range(1, config.EPOCHS + 1):
    train_(config, model, config.DEVICE, train_loader, optimizer, epoch)
    scheduler.step()

NotImplementedError: 